# PROYECTO INTEGRADOR: STEAM
## Dataset -  australian_users_items.json

## 1) Ingesta de los datos

Se descomprimen cada uno de los archivos previo a importarlos con pandas.

In [1]:
#Importamos las librerías que utilizaremos a lo largo de nuestro proyecto
import pandas as pd
from pandas import json_normalize
import json
import ast 

Importamos la información del archivo australian_users_items.json.

In [2]:
# Se carga en un dataframe los items de los usuarios.
users_items = []
with open('Data/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        users_items.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_users_items = pd.DataFrame(users_items)


In [3]:
#Observamos el dataframe
df_users_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [4]:
#Imprimimos las dimensiones del dataframe
df_users_items.shape

(88310, 5)

## 2) Tratamiento de los datos

In [5]:
#Revisamos el dataframe
df_users_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [6]:
#Revisamos la info del dataframe
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [7]:
#Elimino las columnas que no voy a utilizar
df_users_items=df_users_items.drop(columns=['steam_id', 'user_url','items_count'], axis=1)

Queremos desanidar la columna items que está conformada por json anidados.Para eso primero aplicamos el comando explode para expandir o explotar las columnas

In [8]:
# Desagregamos la columna items
df_items_anidado = df_users_items.explode('items')


In [9]:
#Reseteo el index
df_items_anidado.reset_index(drop=True, inplace=True)

In [10]:
#Reviso que se haya realizado la modificación
df_items_anidado.head(5)

,user_id,items
0,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [11]:
#Observo  si hay valores nulos
df_items_anidado.isnull().sum()

user_id        0
items      16806
dtype: int64

Vamos a proceder a hacer el json_normalize para desanidar items, previo a tratar los valores nulos.

In [12]:
#Aplico la normalización a la columna items
df_items_norm = json_normalize(df_items_anidado['items'])

In [13]:
#Reviso que se haya aplanado la columna items
df_items_norm

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...
5170010,373330,All Is Dust,0.0,0.0
5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,519140,Minds Eyes,3.0,3.0


In [14]:
#Elimino la columna playtime_2weeks que no la voy a utilizar
df_items_norm=df_items_norm.drop(columns=['playtime_2weeks'])

In [15]:
#Verifico las dimensiones del dataframe
df_items_norm.shape

(5170015, 3)

In [16]:
#Concateno el dataframe al que le hice el explode con el dataframe al que le apliqué la normalización
df_items=pd.concat([df_items_anidado,df_items_norm], axis=1)

In [17]:
#Verificamos las dimensiones de nuestro dataframe
df_items.shape

(5170015, 5)

In [18]:
#Reviso los nulos nuevamente
df_items.isnull().sum()

user_id                 0
items               16806
item_id             16806
item_name           16806
playtime_forever    16806
dtype: int64

Procedemos a eliminar los nulos. Precisamente en el dataframe items, me interesa tener información sobre los items por lo los registros que solo tienen usuarios, los elimino sino no podré hacer correctamente los merge necesarios para las funciones. 

In [19]:
#Eliminamos los nulos de la columna items
df_items=df_items.dropna(subset='items')

In [20]:
#Verificamos que ya no haya nulos
df_items.isnull().sum()

user_id             0
items               0
item_id             0
item_name           0
playtime_forever    0
dtype: int64

In [21]:
#Revisamos las dimensiones del dataframe
df_items.shape

(5153209, 5)

In [22]:
#Eliminamos la columna items (anidada)
df_items=df_items.drop(columns=['items'])

In [23]:
#Verificamos la eliminación
df_items.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0


Para las funciones solicitadas, se solicita el tiempo de juego por usuario o por género. En este paso y en función de la cantidad de datos presentes, se decide eliminar los valores donde playtime_forever es 0 considerando unicamente las reviews de los usuarios que jugaron al menos 1 hora. El diccionario de datos no determina ninguna cuestión respecto de los usuarios que juegan el juego por menos de una hora, por lo que asumiremos que esos casos son considerados como 0 y también se eliminan.

In [24]:
#Me quedo con los valores donde playtime_forever es distinto de cero
df_items=df_items[df_items['playtime_forever']!=0]

In [25]:
#Reseteamos los indices
df_items.reset_index(drop=True, inplace=True)

In [26]:
#Verificamos nuevamente las dimensiones del dataframe
df_items.shape

(3285246, 4)

In [27]:
#Verificamos el tipo de dato de la columna playtime
df_items['playtime_forever'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3285246 entries, 0 to 3285245
Series name: playtime_forever
Non-Null Count    Dtype  
--------------    -----  
3285246 non-null  float64
dtypes: float64(1)
memory usage: 25.1 MB


In [28]:
#Pasamos de flotante a entero
df_items['playtime_forever'] = df_items['playtime_forever'].astype(int)

In [29]:
#Verificamos la modificación
df_items['playtime_forever'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3285246 entries, 0 to 3285245
Series name: playtime_forever
Non-Null Count    Dtype
--------------    -----
3285246 non-null  int32
dtypes: int32(1)
memory usage: 12.5 MB


In [30]:
#Exportamos a csv
df_items.to_csv('Data/items.csv',index=False)